## Setup

In [133]:
from dataclasses import dataclass

import numpy as np
import pandas
from pathlib import Path
import os

from pandas import DataFrame

from experiment import Protocol
from graph import Graph

In [134]:
@dataclass(frozen=True, order=True)
class Experiment:
    protocol: Protocol
    graph: Graph
    nodes: int
    view_size: int
    shuffle_length: int
    delta_t: int
    disaster_intensity: float

    @classmethod
    def from_strings(cls, protocol, graph, nodes, view_size, shuffle_length, delta_t, disaster_intensity):
        return cls(Protocol(protocol), Graph(graph), int(nodes), int(view_size), int(shuffle_length), int(delta_t), float(disaster_intensity))

In [135]:
runs = {}

for ff in os.listdir(Path() / "runs"):
    if ff.endswith('0.csv'):
        df: DataFrame = pandas.read_csv(Path() / "runs" / ff)
        df.drop(df.columns[0], axis=1, inplace=True)
        rows, _ = df.shape
        if rows > 0:
            params = tuple(ff.rstrip(' 0.csv').split(' '))
            runs[Experiment.from_strings(*params)] = df
print(f"There are {len(runs)} saved runs.")

There are 433 saved runs.


## Partitioning

In [136]:
partitioned_runs = {}

for run in runs:
    df = runs[run]
    rows, _ = df.shape
    if max(df.head(rows // 3)["Alive Partitions"]) > 1:
        partitioned_runs[run] = df

In [137]:
print(f"There are {len(partitioned_runs)} partitioned runs:")
for params in partitioned_runs:
    print(f"  - {params}")

There are 20 partitioned runs:
  - Experiment(protocol=<Protocol.CYCLON: 'cyclon'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=100, shuffle_length=10, delta_t=1, disaster_intensity=0.5)
  - Experiment(protocol=<Protocol.CYCLON: 'cyclon'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=100, shuffle_length=10, delta_t=1, disaster_intensity=0.75)
  - Experiment(protocol=<Protocol.CYCLON: 'cyclon'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=100, shuffle_length=10, delta_t=1, disaster_intensity=0.95)
  - Experiment(protocol=<Protocol.CYCLON: 'cyclon'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=20, shuffle_length=10, delta_t=1, disaster_intensity=0.5)
  - Experiment(protocol=<Protocol.CYCLON: 'cyclon'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=20, shuffle_length=10, delta_t=1, disaster_intensity=0.75)
  - Experiment(protocol=<Protocol.CYCLON: 'cyclon'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=20, shuffle_length=15, delta_t=1, disaster_intensity=0.5)
  - Exper

In [138]:
recovered_runs = {}

for run in partitioned_runs:
    df = partitioned_runs[run]
    rows, _ = df.shape
    if df["Alive Partitions"][rows // 3] == 1:
        recovered_runs[run] = df

In [139]:
print(f"There are {len(recovered_runs)} recovered runs:")
for params in partitioned_runs:
    print(f"  - {params}")

There are 18 recovered runs:
  - Experiment(protocol=<Protocol.CYCLON: 'cyclon'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=100, shuffle_length=10, delta_t=1, disaster_intensity=0.5)
  - Experiment(protocol=<Protocol.CYCLON: 'cyclon'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=100, shuffle_length=10, delta_t=1, disaster_intensity=0.75)
  - Experiment(protocol=<Protocol.CYCLON: 'cyclon'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=100, shuffle_length=10, delta_t=1, disaster_intensity=0.95)
  - Experiment(protocol=<Protocol.CYCLON: 'cyclon'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=20, shuffle_length=10, delta_t=1, disaster_intensity=0.5)
  - Experiment(protocol=<Protocol.CYCLON: 'cyclon'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=20, shuffle_length=10, delta_t=1, disaster_intensity=0.75)
  - Experiment(protocol=<Protocol.CYCLON: 'cyclon'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=20, shuffle_length=15, delta_t=1, disaster_intensity=0.5)
  - Experim

In [140]:
unrecovered_runs = set(partitioned_runs.keys()) - set(recovered_runs.keys())
print(f"There are {len(unrecovered_runs)} unrecovered runs.")
for run in sorted(unrecovered_runs):
    print(f"  - {run}")

There are 2 unrecovered runs.
  - Experiment(protocol=<Protocol.NEWSCAST: 'newscast'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=20, shuffle_length=10, delta_t=4, disaster_intensity=0.5)
  - Experiment(protocol=<Protocol.NEWSCAST: 'newscast'>, graph=<Graph.GEO: 'geo'>, nodes=1000, view_size=20, shuffle_length=10, delta_t=4, disaster_intensity=0.75)


## Plots

In [162]:
# Clustering coefficients
groups = []

for proto in (Protocol.CYCLON, Protocol.NEWSCAST):
    for view_size in (20, 50, 100):
        for graph in (Graph.GEO, Graph.RANDOM, Graph.LATTICE, Graph.STAR):
            groups.append(sorted(list(
                filter(
                    lambda _: (
                        _.protocol == proto
                        and _.view_size == view_size
                        and _.graph == graph
                        # and _ not in partitioned_runs
                    ),
                    runs,
                )
            )))

for group in groups:
    group_average = DataFrame(columns=["CC"])

    for experiment in group:
        normalized = np.zeros(301)
        DT = experiment.delta_t
        for cycle in range(301):
            normalized[cycle] = sum(
                runs[experiment]["Alive Clustering Coefficient"][cycle*DT: cycle+DT]
            ) / experiment.delta_t
        if len(group_average) == 0:
            group_average["CC"] = normalized
        else:
            group_average["CC"] += normalized
            group_average["CC"] /= 2

    with open(Path() / "report" / "figures" / f"clustering_coefficient_{experiment.protocol.value}_{experiment.view_size}_{experiment.graph.value}.dat", 'w') as f:
        f.write("step CC\n")
        f.writelines([
            f"{i} {r}\n"
            for i, r in enumerate(group_average["CC"])
        ])
        print(f"Written {f.name}.")

Written report\figures\clustering_coefficient_cyclon_20_geo.dat.
Written report\figures\clustering_coefficient_cyclon_20_random.dat.
Written report\figures\clustering_coefficient_cyclon_20_lattice.dat.
Written report\figures\clustering_coefficient_cyclon_20_star.dat.
Written report\figures\clustering_coefficient_cyclon_50_geo.dat.
Written report\figures\clustering_coefficient_cyclon_50_random.dat.
Written report\figures\clustering_coefficient_cyclon_50_lattice.dat.
Written report\figures\clustering_coefficient_cyclon_50_star.dat.
Written report\figures\clustering_coefficient_cyclon_100_geo.dat.
Written report\figures\clustering_coefficient_cyclon_100_random.dat.
Written report\figures\clustering_coefficient_cyclon_100_lattice.dat.
Written report\figures\clustering_coefficient_cyclon_100_star.dat.
Written report\figures\clustering_coefficient_newscast_20_geo.dat.
Written report\figures\clustering_coefficient_newscast_20_random.dat.
Written report\figures\clustering_coefficient_newscast_